In [27]:
import mysql.connector
import pandas as pd

In [28]:
def connect():
  connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "wp-ecommerce"
  )

  cursor = connection.cursor(dictionary = True)

  return connection, cursor

In [29]:
def get_product_title(product_id):
  _, cursor = connect()

  sql = '''
    SELECT post_title FROM wp_posts WHERE ID = (%s)
  '''
  
  cursor.execute(sql, (product_id, ))
  
  result = cursor.fetchone()

  return result["post_title"] if result else None

In [30]:
get_product_title(55945)

"بنطال قماش رجالي-أزرق-Levi's"

In [31]:
def build_transactions():
  _, cursor = connect()

  sql = '''
    SELECT * FROM wp_wc_order_stats ORDER BY order_id
  '''
  
  cursor.execute(sql)

  results = cursor.fetchall()

  data = pd.DataFrame(columns = [0, 1, 2, 3, 4])
  
  for row in results:
    order_id = row["order_id"]

    sql = '''
      SELECT * FROM wp_wc_order_product_lookup WHERE order_id = (%s)
    '''
  
    cursor.execute(sql, (order_id, ))
    
    order_products = cursor.fetchall()

    product_ids = []

    for product in order_products:
      product_id = product["product_id"]

      if product_id > 0:
        product_ids.append(product_id)

    if len(product_ids) > 1:
      data = pd.concat([data, pd.DataFrame([product_ids])], ignore_index = True)
  
  return data

In [32]:
transactions = build_transactions()

transactions

,0,1,2,3,4
0,55959,55979,56265,55968,NaN
1,55945,55947,56108,56261,55973
2,55890,55931,56064,56297,55898
3,56007,55982,56152,56286,55981
4,55884,55886,56082,56288,55879
...,...,...,...,...,...
2540,56214,56247,56238,56235,NaN
2541,56354,56341,NaN,NaN,NaN
2542,55958,56005,NaN,NaN,NaN
2543,56007,56115,55976,NaN,NaN


In [33]:
from mlxtend.preprocessing import TransactionEncoder

def encode_transactions(transactions):
  transactions = transactions.T

  transactions = transactions.apply(lambda x: x.dropna().tolist())

  transactions = transactions.tolist()

  encoder = TransactionEncoder()

  encoded_transactions = encoder.fit_transform(transactions)

  data = pd.DataFrame(encoded_transactions, columns = encoder.columns_)

  return data

In [34]:
transactions = encode_transactions(transactions)

transactions

,55868,55869,55870,55873,55874,55876,55877,55878,55879,55881,...,56345,56347,56350,56351,56354,56355,56358,56359,56360,56363
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2543,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def generate_association_rules(transactions, min_support, min_confidence):
  frequent_itemsets = apriori(transactions, min_support = min_support, use_colnames = True)
  frequent_itemsets["length"] = frequent_itemsets["itemsets"].apply(lambda x: len(x))

  rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = min_confidence)
  rules = rules.sort_values("confidence", ascending = False)

  return rules

#### Try to find the best possible min_support value

In [36]:
# set initial min_support value that returns no results
min_support = 0.01

# 0.1 is an experimental value
rules = generate_association_rules(transactions, min_support, 0.1)

len(rules)

0

In [37]:
steps = 0
count = 0

# 50 is how many rules we would like to generate
while count < 50:
  min_support = min_support * 90 / 100
  steps += 1

  rules = generate_association_rules(transactions, min_support, 0.1)
  count = len(rules)

  print(f"min_support: {min_support}, rules count: {count}")


min_support: 0.009000000000000001, rules count: 0
min_support: 0.008100000000000001, rules count: 0
min_support: 0.007290000000000001, rules count: 0
min_support: 0.006561000000000001, rules count: 2
min_support: 0.005904900000000001, rules count: 8
min_support: 0.00531441, rules count: 22
min_support: 0.004782969000000001, rules count: 26
min_support: 0.0043046721, rules count: 52


In [38]:
rules = generate_association_rules(transactions, 0.004, 0.1)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15,(56235),(56193),0.009430,0.031434,0.004322,0.458333,14.580729,0.004026,1.788122,0.940283
28,(56209),(56238),0.012181,0.036149,0.005501,0.451613,12.492987,0.005061,1.757610,0.931299
51,(56323),(56305),0.010609,0.016896,0.004322,0.407407,24.112834,0.004143,1.658988,0.968806
4,(56179),(56238),0.017289,0.036149,0.006287,0.363636,10.059289,0.005662,1.514623,0.916433
1,(56179),(56218),0.017289,0.029862,0.006287,0.363636,12.177033,0.005771,1.524502,0.934026
20,(56195),(56238),0.015717,0.036149,0.005501,0.350000,9.682065,0.004933,1.482847,0.911035
40,(56227),(56228),0.014931,0.031041,0.004715,0.315789,10.173218,0.004252,1.416170,0.915370
8,(56188),(56228),0.017682,0.031041,0.005501,0.311111,10.022504,0.004952,1.406553,0.916429
35,(56245),(56217),0.014145,0.033399,0.004322,0.305556,9.148693,0.003850,1.391906,0.903475
42,(56232),(56228),0.015717,0.031041,0.004715,0.300000,9.664557,0.004227,1.384227,0.910845


In [39]:
def predict(rules, items, max_results = 6):
  predicts = rules[rules["antecedents"] == items]
  predicts = predicts[["consequents", "confidence"]]
  predicts.sort_values("confidence", ascending = False)

  return predicts[:max_results]

In [40]:
predictions = predict(rules, {56193})

predictions

,consequents,confidence
10,(56228),0.2000
16,(56247),0.1625
12,(56230),0.1500
14,(56235),0.1375


In [41]:
print(f"Item: {get_product_title(56193)}\n")

print("Recommendations:")
for _, prediction in predictions.iterrows():
  for product_id in prediction["consequents"]:
    print(get_product_title(product_id))

Item: فرن-بني-HP

Recommendations:
غسالة-أسود-LG
غسالة-فضي-SONY
غسالة-أسود-SONY
مصفف شعر-أسود داكن-SONY


In [42]:
def export_associations_rules(rules):
  connection, cursor = connect()

  sql = "DROP TABLE IF EXISTS wp_wc_product_associations"
  
  cursor.execute(sql)

  sql = '''
    CREATE TABLE wp_wc_product_associations (
      ID int(11) NOT NULL AUTO_INCREMENT,
      antecedent_product_id int(11) NOT NULL,
      antecedent_product_title text NOT NULL,
      consequent_product_id int(11) NOT NULL,
      consequent_product_title text NOT NULL,
      confidence double NOT NULL,
      PRIMARY KEY (ID)
    )
  '''
  
  cursor.execute(sql)
  
  connection.commit()

  for rule in rules.itertuples():
    antecedents = rule.antecedents
    consequents = rule.consequents
    confidence = rule.confidence * 100

    for antecedent_product_id in antecedents:
      antecedent_product_title = get_product_title(antecedent_product_id)

      for consequent_product_id in consequents:
        consequent_product_title = get_product_title(consequent_product_id)
        
        # delete association rule with low confidence
        sql = '''
          DELETE FROM wp_wc_product_associations
          WHERE antecedent_product_id = (%s)
          AND consequent_product_id = (%s)
          AND confidence < (%s)
        '''
  
        cursor.execute(sql, (antecedent_product_id, consequent_product_id, confidence))

        # get association rule with accepted or better confidence
        sql = '''
          SELECT * FROM wp_wc_product_associations
          WHERE antecedent_product_id = (%s)
          AND consequent_product_id = (%s)
          AND confidence >= (%s)
        '''
  
        cursor.execute(sql, (antecedent_product_id, consequent_product_id, confidence))

        result = cursor.fetchone()

        # insert association rule if not stored in the table
        insert_association = False if result else True

        if insert_association:
          sql = '''
            INSERT INTO wp_wc_product_associations VALUES (NULL, %s, %s, %s, %s, %s)
          '''
  
          cursor.execute(sql, (antecedent_product_id, antecedent_product_title,
                              consequent_product_id, consequent_product_title,
                              confidence))
    
          connection.commit()

In [43]:
export_associations_rules(rules)

In [44]:
def get_recommended_products_by_product(product_id):
  _, cursor = connect()

  sql = '''
    SELECT * FROM wp_wc_product_associations
    WHERE antecedent_product_id = (%s)
    ORDER BY confidence DESC
  '''
  
  cursor.execute(sql, (product_id, ))

  results = cursor.fetchall()

  entries = []

  for row in results:
    entries.append({
      "product_id": row["consequent_product_id"],
      "product_title": row["consequent_product_title"],
      "confidence": round(row["confidence"], 2)
    })
  
  data = pd.DataFrame(entries)

  return data

In [45]:
data = get_recommended_products_by_product(56193)

data

,product_id,product_title,confidence
0,56228,غسالة-أسود-LG,20.00
1,56247,غسالة-فضي-SONY,16.25
2,56230,غسالة-أسود-SONY,15.00
3,56235,مصفف شعر-أسود داكن-SONY,13.75
